In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt
import Fvvsc

## 1. Create function for calculating entropy density
- inputs: array for energy values (q) and two density matrices for neutrinos and anti neutrinos
- the function uses trapezoid rule to calculate the entropy density integral: 
    - for each density matrix calculate the eigenvalues
    - use eigenvalues to calculate tr[$\rho$ ln $\rho$]
- use data file to calculate s at every time step

$s = -\int_{0}^{\infty} q^2 \,dq \, \mathrm{trc} [\rho_q\mathrm{ln}\rho_q]$

In [2]:
## my trapezoid rule from dm

#@nb.jit(nopython=True)
def trapezoid(x,y): # x is array of indep vars and y is array of integrand values
    N=len(x)
    summ = 0
    for i in range(N-1):
        I = 0.5*(y[i]+ y[i+1])*(x[i+1] - x[i])
        summ = summ + I
    return summ

In [3]:
old_data = np.load('Run35.npz', allow_pickle = True)
data = np.load('1000keV-run_long.npz', allow_pickle = True)
data_col = np.load('1000keV-run_long-Collision.npz', allow_pickle = True)

In [4]:
#array for energy values
settings = data['settings'].item()
T = settings['T']
N = settings['N'] + 1 #=201
emax = settings['eps_max']
q_vals = np.linspace(0, emax*T, N)

#Matrices for rho and rho bar
rawdata = data['raw'].item()
nu3D = rawdata['nu3D']
nubar3D = rawdata['nubar3D']

nu3D.shape[1]
q_vals.shape[0]
rawdata.keys()

dict_keys(['initial array', 'time', 'Ht', 'dt', 'dN', 'nu3D', 'nubar3D', 'nu_init matrix', 'nu_bar_init matrix'])

In [5]:
def solve_entropy(q_vals, rho, rhobar): #rho and rhobar are 2 dimensional
    #initialize
    N = q_vals.shape[0] # this is now 201, starting at zero
    traces = [0]
    tracesbar = [0]

    for i in range(N-1): # for every matrix in each time step
        # get matrix into components
        P0, Px, Py, Pz = rho[i,:] 
    
        # calculate eigenvalues
        p_mag = np.sqrt(Px**2 + Py**2 + Pz**2)
        lambda1 = 0.5*P0 + 0.5*P0*p_mag
        lambda2 = 0.5*P0 - 0.5*P0*p_mag
        klnk1 = lambda1*np.log(lambda1) if lambda1 > 0 else 0
        klnk2 = lambda2*np.log(lambda2) if lambda2 > 0 else 0
        trace = (klnk1 + klnk2) * q_vals[i+1]**2
        # or q_vals[i+1]**2?????
    
        #now do it all again for nubar
        P0bar, Pxbar, Pybar, Pzbar = rhobar[i,:]
        pbar_mag = np.sqrt(Pxbar**2 + Pybar**2 + Pzbar**2)
        lambda1bar = 0.5*P0bar + 0.5*P0bar*pbar_mag
        lambda2bar = 0.5*P0bar - 0.5*P0bar*pbar_mag
        klnk1bar = lambda1bar*np.log(lambda1bar) if lambda1bar > 0 else 0
        klnk2bar = lambda2bar*np.log(lambda2bar) if lambda2bar > 0 else 0
        tracebar = (klnk1bar + klnk2bar) * q_vals[i+1]**2
#NOTE: for this to work with my trap function, trace and tracebar need to be arrays, so at the end of every loop here
# i should save the trace and tracebar values into an array and then i could put those into trap
        #ok so after each calculation of trace and tracebar, ill append them to a list, then i can put those lits into trapezoid instead
        traces.append(trace)
        tracesbar.append(tracebar)
    
        trace_vals = np.array(traces)
        tracebar_vals = np.array(tracesbar)

        #calculate s
    s = -1/(2*np.pi**2)*(trapezoid(q_vals[:len(trace_vals)], trace_vals) + trapezoid(q_vals[:len(trace_vals)], tracebar_vals))
    return s

## 2. Use data file to calculate s

In [11]:
s = np.zeros((N)) 

for j in range(N):
    s[j] = np.array([solve_entropy(q_vals, nu3D[j,:,:], nubar3D[j,:,:])])
    print(s[j])

1.1765654796620373
1.176565479662032
1.1765654796620244
1.176565479662015
1.1765654796620055
1.1765654796619984
1.1765654796619915
1.1765654796619864
1.176565479661982
1.1765654796619762
1.1765654796619711
1.1765654796619638
1.1765654796619556
1.1765654796619474
1.1765654796619374
1.1765654796619258
1.176565479661914
1.1765654796619014
1.176565479661888
1.1765654796618732
1.1765654796618579
1.1765654796618412
1.1765654796618215
1.1765654796618006
1.1765654796617788
1.176565479661755
1.1765654796617289
1.1765654796617029
1.1765654796616718
1.1765654796616392
1.176565479661604
1.176565479661566
1.1765654796615288
1.1765654796614826
1.1765654796614369
1.1765654796613894
1.1765654796613336
1.1765654796612754
1.1765654796612155
1.1765654796611538
1.1765654796610854
1.1765654796610066
1.1765654796609233
1.1765654796608365
1.176565479660747
1.1765654796606566
1.1765654796605638
1.1765654796604608
1.1765654796603393
1.1765654796602052
1.1765654796600633
1.1765654796599103
1.1765654796597511
1.

## 3. Calculate derivative $\frac{1}{s}\frac{ds}{dt} = \frac{d \mathrm{ln}s}{dt}$ approximate as $\approx \frac{\mathrm{ln}s(t + dt) - \mathrm{ln}s(t)}{dt}$
- to calculate $s(t + dt)$ need to evolve $(P_0, \vec{P})$ from $t$ to $t+dt$ for each of our density matrices

In [8]:
data_col = np.load('1000keV-run_long-Collision.npz', allow_pickle = True)
data_col.files
C_vals = data_col['coll_nu']
C_bar_vals = data_col['coll_nubar']

In [9]:
def dsdt(q_vals, rho, rho_bar, col, col_bar): #rho and rhobar are 2 dimensional
    #initialize
    N = q_vals.shape[0] # this is now 201, starting at zero
    dtraces = [0]
    dtraces_bar = [0]

    for i in range(N-1): # for every matrix in each time step
        P0, Px, Py, Pz = rho[i,:]
        C0, Cx, Cy, Cz = col[i,:]
        pmag = np.sqrt(Px**2 + Py**2 + Pz**2)
        lambda1 = 0.5*P0 + 0.5*P0*pmag
        lambda2 = 0.5*P0 - 0.5*P0*pmag
        
        #new derivaite stuff
        dP0dt = 2*C0
        dpmagdt = (1/2*pmag)*(-2*(C0/P0)*pmag**2 + (4/P0)*(Cx*Px + Cy*Py + Cz*Pz))

        dk1dt = 0.5*dP0dt*(1+pmag) + 0.5*P0*dpmagdt
        dk2dt = 0.5*dP0dt*(1-pmag) - 0.5*P0*dpmagdt
        
        dklnk1 = dk1dt*(np.log(lambda1)+1) if lambda1 > 0 else 0
        dklnk2 = dk2dt*(np.log(lambda2)+1) if lambda2 > 0 else 0
        dtrace = (dklnk1 + dklnk2) * q_vals[i+1]**2
    
#now do it all again for nubar
        P0_bar, Px_bar, Py_bar, Pz_bar = rho_bar[i,:]
        C0_bar, Cx_bar, Cy_bar, Cz_bar = col_bar[i,:]
        pmag_bar = np.sqrt(Px_bar**2 + Py_bar**2 + Pz_bar**2)
        lambda1_bar = 0.5*P0_bar + 0.5*P0_bar*pmag_bar
        lambda2_bar = 0.5*P0_bar - 0.5*P0_bar*pmag_bar
        
        #new derivaite stuff
        dP0dt_bar = 2*C0_bar
        dpmagdt_bar = (1/2*pmag_bar)*(-2*(C0_bar/P0_bar)*pmag_bar**2 + (4/P0_bar)*(Cx_bar*Px_bar + Cy_bar*Py_bar + Cz_bar*Pz_bar))

        dk1dt_bar = 0.5*dP0dt_bar*(1+pmag_bar) + 0.5*P0*dpmagdt_bar
        dk2dt_bar = 0.5*dP0dt_bar*(1-pmag_bar) - 0.5*P0*dpmagdt_bar
        
        dklnk1_bar = dk1dt_bar*(np.log(lambda1_bar)+1) if lambda1_bar > 0 else 0
        dklnk2_bar = dk2dt_bar*(np.log(lambda2_bar)+1) if lambda2_bar > 0 else 0      
        dtrace_bar = (dklnk1_bar + dklnk2_bar) * q_vals[i+1]**2
        
        dtraces.append(dtrace)
        dtraces_bar.append(dtrace_bar)
        dtrace_vals = np.array(dtraces)
        dtrace_vals_bar = np.array(dtraces_bar)

    #calculate s
    s = -1/(2*np.pi**2)*(trapezoid(q_vals[:len(dtrace_vals)], dtrace_vals) + trapezoid(q_vals[:len(dtrace_vals)], dtrace_vals_bar))
    return s

In [10]:
ds = np.zeros((N)) 

for j in range(N):
    ds[j] = np.array([dsdt(q_vals, nu3D[j,:,:], nubar3D[j,:,:], C_vals[j,:,:], C_bar_vals[j,:,:])])
    print(ds[j])

-1.2136538363209769e-23
-1.2136538345480957e-23
-1.2136538330935919e-23
-1.2136538331290447e-23
-1.2136538337713758e-23
-1.2136538345130491e-23
-1.2136538341876345e-23
-1.2136538334311808e-23
-1.2136538332506124e-23
-1.2136538333563869e-23
-1.2136538339566507e-23
-1.2136538341845255e-23
-1.2136538338347481e-23
-1.2136538333175913e-23
-1.2136538334098134e-23
-1.213653833688076e-23
-1.2136538339485827e-23
-1.2136538339406928e-23
-1.2136538334945996e-23
-1.213653833610957e-23
-1.2136538336250639e-23
-1.2136538341007647e-23
-1.2136538340885969e-23
-1.2136538338926072e-23
-1.2136538336538684e-23
-1.2136538336269634e-23
-1.2136538338810141e-23
-1.2136538338122326e-23
-1.2136538339447702e-23
-1.2136538335900797e-23
-1.2136538335712611e-23
-1.2136538339928564e-23
-1.2136538339138141e-23
-1.2136538337776923e-23
-1.2136538336725089e-23
-1.213653833907201e-23
-1.2136538338346822e-23
-1.2136538337200722e-23
-1.2136538336810687e-23
-1.2136538336417384e-23
-1.2136538338607166e-23
-1.2136538339301326